In [1]:
from fastapi import FastAPI, HTTPException
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import re
from openai import OpenAI
import anthropic
import os
from fastapi.responses import JSONResponse
import uuid
import requests
import json


In [2]:
OPEN_API_KEY = os.getenv("OPEN_API_KEY")
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
POSTGRES_USER=os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD=os.getenv("POSTGRES_PASSWORD")
POSTGRES_DB=os.getenv("POSTGRES_DB")
POSTGRES_HOST=os.getenv("POSTGRES_HOST")
POSTGRES_PORT=os.getenv("POSTGRES_PORT")

In [3]:
from openai import OpenAI
import numpy as np

openai_client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=OPEN_API_KEY
)

anthropic_client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

## Data Load

In [4]:
# GET COMPELETE PRODUCT DATASET
def get_products():
    """
    Fetches all product details from the external API.

    Args:
        No ARGS

    Returns:
        A dictionary containing product details.
    """
    try:
        response = requests.get(f"http://localhost:8000/products")
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching product details: {e}")
        return {"error": f"No Product availabel"}

In [5]:
# GET COMPELETE HISTORY OF ORDER BY ORDER ID
def get_orders(order_id: str):
    """
    Fetches order details from the external API.

    Args:
        order_id: The unique identifier of the order.

    Returns:
        A dictionary containing order details and shipment history.
    """
    try:
        response = requests.get(f"http://localhost:8000/orders/{order_id}")
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching order details: {e}")
        return {"error": f"Failed to fetch shipment details for order ID {order_id}"}

In [6]:
# GET COMPELETE HISTORY OF SHIPMENT BY SHIPMENT ID
def get_shipment_history(shipment_id: str):
    """
    Fetches shipment details from the external API.

    Args:
        shipment_id: The unique identifier of the shipment.

    Returns:
        A dictionary containing order details and shipment history.
    """
    try:
        response = requests.get(f"http://localhost:8000/shipments/{shipment_id}")
        response.raise_for_status()  # Raise an exception for bad status codes
        return [response.json()]
    except requests.exceptions.RequestException as e:
        print(f"Error fetching order details: {e}")
        return {"error": f"Failed to fetch shipment details for shipment ID {shipment_id}"}

In [7]:
# GET COMPELETE HISTORY OF SHIPMENT BY ORDER ID
def get_order_with_shipment_history(order_id: str):
    """
    Fetches order details from the external API.

    Args:
        order_id: The unique identifier of the order.

    Returns:
        A dictionary containing order details and shipment history.
    """
    try:
        response = requests.get(f"http://localhost:8000/shipment_details/{order_id}")
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching order details: {e}")
        return {"error": f"Failed to fetch order details for order ID {order_id}"}

In [8]:
get_products()

[{'product_name': 'Apple iPhone 15 Pro',
  'description': 'The latest flagship smartphone from Apple, featuring a powerful A17 Bionic chip, a stunning Super Retina XDR display, and a triple-lens camera system.',
  'usage': 'Communication, entertainment, productivity, and more.',
  'summary': 'The iPhone 15 Pro is a premium smartphone that offers top-of-the-line performance, features, and design.',
  'cost': 999.0},
 {'product_name': 'Sony WH-1000XM5 Noise-Canceling Headphones',
  'description': 'Industry-leading noise-canceling headphones with exceptional sound quality and comfortable fit.',
  'usage': 'Listening to music, watching movies, and making calls.',
  'summary': 'The Sony WH-1000XM5 headphones are a great choice for anyone who wants the best possible noise-canceling and sound quality.',
  'cost': 399.0},
 {'product_name': 'Dyson V15 Detect Cordless Vacuum',
  'description': 'A powerful cordless vacuum with advanced laser dust detection and self-adjusting suction.',
  'usage':

In [9]:
knowledge_base = get_products()

In [10]:
tools = [
        {
        "name": "get_orders",
        "description": "This function is used to provide details of a specific order by its unique Order ID. \
                        It includes details like User ID, Product Name, Cost, Quantity of order, Order Date, Shipment ID and current Status Of Order.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The Order ID of the order."
                }
            },
            "required": ["order_id"]
        }
    },
    {
        "name": "get_shipment_history",
        "description": "This function is used to provide complete details of shipment history for a Shipment ID. \
            It helps to track an order from the date it is ordered and different stages of shippment involved ducring package shipments. \
            It also provides each stages of shipment like packed, dispatched, out for delivery, delivered, and returned along with date.",
        "input_schema": {
            "type": "object",
            "properties": {
                "shipment_id": {
                    "type": "string",
                    "description": "The Shipment ID of the shipment."
                }
            },
            "required": ["shipment_id"]
        }
    },
    {
        "name": "get_order_with_shipment_history",
        "description": "This function is used to provide complete details of Order and shipment history for a Provide Order ID.",
        "input_schema": {
            "type": "object",
            "properties": {
                "order_id": {
                    "type": "string",
                    "description": "The Order ID of the order."
                }
            },
            "required": ["order_id"]
        }
    }
]

In [11]:
async def process_tool_call(tool_name, tool_input):
    if tool_name == "get_products":
        return get_products(tool_input[""])
    elif tool_name == "get_orders":
        return get_orders(tool_input["order_id"])
    elif tool_name == "get_shipment_history":
        return get_shipment_history(tool_input["shipment_id"])
    elif tool_name == "get_order_with_shipment_history":
        return get_order_with_shipment_history(tool_input["order_id"])
    else:
        raise ValueError(f"Unknown tool: {tool_name}")

## CHAT BOT

### FUNCTION ONLY

In [12]:
import json

async def chatbot_interaction(user_message):
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

    messages = [
        {"role": "user", "content": user_message}
    ]

    # Mocked `anthropic_client` for demonstration purposes
    response = anthropic_client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        tools=tools,
        messages=messages
    )

    print(f"\nInitial Response:")
    print(f"Stop Reason: {response.stop_reason}")
    print(f"Content: {response.content}")

    while response.stop_reason == "tool_use":
        tool_use = next(block for block in response.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        print(f"\nTool Used: {tool_name}")
        print(f"Tool Input:")
        print(json.dumps(tool_input, indent=2))

        tool_result = await process_tool_call(tool_name, tool_input)

        print(f"\nTool Result:")
        print(json.dumps(tool_result, indent=2))

        messages = [
            {"role": "user", "content": user_message},
            {"role": "assistant", "content": response.content},
            {
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,
                        "content": str(tool_result),
                    }
                ],
            },
        ]

        # Mocked `anthropic_client` for demonstration purposes
        response = anthropic_client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        tools=tools,
        messages=messages
        )

        print(f"\nResponse:")
        print(f"Stop Reason: {response.stop_reason}")
        print(f"Content: {response.content}")

    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )

    print(f"\nFinal Response: {final_response}")

    return final_response

In [13]:
await chatbot_interaction("Can you tell me the shipment details with complete tracker for ORD1001")
# await chatbot_interaction("What is the status of order ORD1001?")
# await chatbot_interaction("What is Apple iPhone 15 Pro used for")


User Message: Can you tell me the shipment details with complete tracker for ORD1001

Initial Response:
Stop Reason: tool_use
Content: [TextBlock(text='To get the complete shipment history and tracker for the order with ID ORD1001, we can use the "get_order_with_shipment_history" tool:', type='text'), ToolUseBlock(id='toolu_01PYpDQ1QNcJBQkNd3FUkbv1', input={'order_id': 'ORD1001'}, name='get_order_with_shipment_history', type='tool_use')]

Tool Used: get_order_with_shipment_history
Tool Input:
{
  "order_id": "ORD1001"
}

Tool Result:
{
  "order": {
    "order_id": "ORD1001",
    "userid": 1,
    "product": "Laptop",
    "cost": 1200.0,
    "quantity": 1,
    "order_date": "2024-12-20",
    "shipment_id": "SHP123",
    "order_status": "Shipped"
  },
  "shipment_details": [
    {
      "shipment_id": "SHP123",
      "pin_code": "123456",
      "shipment_status": "Shipped",
      "last_update": "2024-12-20",
      "comments": "Package has been dispatched."
    },
    {
      "shipment_id

'This provides all the order details like product, cost, quantity, date as well as the complete shipment history tracker showing the status updates from dispatch to out for delivery to finally delivered along with dates.'

### Function and Knowledge Base

In [14]:
async def anthropic_knowledge_function_call(user_query, knowledge_base, tools):
    prompt = f"""
    You are a support assistant. 
    Your job is to answer user queries based on the following knowledge base and use available tools when required.
    Whenever customers enquires about their order status, ask for Order ID or Shipment ID if it is not available in the conversation.

    Star with the summary of their order in few lines, and provide details of the ir order in new section.

    Knowledge Base:
    {knowledge_base}

    Tools:
    {json.dumps(tools, indent=2)}

    User Query:
    {user_query}

    If the answer is directly available in the knowledge base, respond with it. 
    If you need to use a tool, specify the tool and its parameters.
    Provide answers in readbale and formatted text.
    """
    history = []

    messages = [
        {"role": "user", "content": prompt}
    ]

    # Mocked `anthropic_client` for demonstration purposes
    response = anthropic_client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        tools=tools,
        messages=messages
    )

    # print(f"\nInitial Response:")
    # print(f"Stop Reason: {response.stop_reason}")
    # print(f"Content: {response.content}")

    while response.stop_reason == "tool_use":
        tool_use = next(block for block in response.content if block.type == "tool_use")
        tool_name = tool_use.name
        tool_input = tool_use.input

        # print(f"\nTool Used: {tool_name}")
        # print(f"Tool Input:")
        # print(json.dumps(tool_input, indent=2))

        tool_result = await process_tool_call(tool_name, tool_input)

        # print(f"\nTool Result:")
        # print(json.dumps(tool_result, indent=2))

        messages = [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": response.content},
            {
                "role": "user",
                "content": [
                    {
                        "type": "tool_result",
                        "tool_use_id": tool_use.id,
                        "content": str(tool_result),
                    }
                ],
            },
        ]

        # Mocked `anthropic_client` for demonstration purposes
        response = anthropic_client.messages.create(
        model="claude-3-sonnet-20240229",
        max_tokens=4096,
        tools=tools,
        messages=messages
        )

        # print(f"\nResponse:")
        # print(f"Stop Reason: {response.stop_reason}")
        # print(f"Content: {response.content}")

    final_response = next(
        (block.text for block in response.content if hasattr(block, "text")),
        None,
    )

    # print(f"\nFinal Response: {final_response}")

    # history.append((user_query, final_response))

    return final_response


In [15]:
user_query = "Provide me complete order shipment details of ORD1001, also provide some usage of the product"
result = await anthropic_knowledge_function_call(user_query, knowledge_base, tools)
print(result)

Summary:
The order ORD1001 was placed by User ID 1 for a Laptop costing $1200. The order was shipped with Shipment ID SHP123 on 2024-12-20. The shipment was delivered successfully on 2024-12-21.

Order and Shipment Details:

Order ID: ORD1001
User ID: 1
Product: Laptop  
Cost: $1200
Quantity: 1
Order Date: 2024-12-20
Shipment ID: SHP123
Order Status: Shipped

Shipment History:

Shipment ID: SHP123
Pincode: 123456
Status: Shipped 
Last Update: 2024-12-20
Comments: Package has been dispatched.

Shipment ID: SHP123  
Pincode: 123456
Status: Out for Delivery
Last Update: 2024-12-21  
Comments: Package is out for delivery.

Shipment ID: SHP123
Pincode: 123456  
Status: Delivered
Last Update: 2024-12-21
Comments: Package delivered successfully.

Usage:
A laptop can be used for various purposes such as communication, entertainment, productivity tasks like document editing, data analysis, coding, and more.


In [16]:
# !pip install gradio

In [17]:
import gradio as gr
import json

# Gradio async wrapper
async def gradio_chatbot(user_query, history):
    response = await anthropic_knowledge_function_call(user_query, knowledge_base, tools)
    history = history or []
    history.append((user_query, response))
    return history, history

# Gradio interface setup
with gr.Blocks() as demo:
    chatbot_ui = gr.Chatbot(label="AI Chatbot with Tools")
    message = gr.Textbox(label="Your Query", placeholder="Ask me anything...", lines=1)
    state = gr.State()  # For chat history
    send_button = gr.Button("Send")

    send_button.click(
        fn=gradio_chatbot,
        inputs=[message, state],
        outputs=[chatbot_ui, state],
        api_name="chat"
    )

# demo.launch()
# Launch the app
demo.launch(server_port=7860)


/opt/anaconda3/envs/trainenv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/trainenv/lib/python3.12/site-packages/gradio/components/chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Error fetching order details: 404 Client Error: Not Found for url: http://localhost:8000/shipments/TST9090
Error fetching order details: 404 Client Error: Not Found for url: http://localhost:8000/orders/TMT1001
Error fetching order details: 404 Client Error: Not Found for url: http://localhost:8000/shipment_details/ORDT1001
Error fetching order details: 404 Client Error: Not Found for url: http://localhost:8000/shipment_details/ORDT1001
